<a href="https://colab.research.google.com/github/andgreenman/OpenAlex-removing-unaffiliated-authors/blob/main/OpenAlex_removing_unaffiliated_authors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script takes an OpenAlex search export csv and deletes all listed authors without a specified affiliation for the purposes of using Biblioshiny to create a network graph and examine collaboration solely between authors at one institution. VOSviewer can also be used with further processing, found in another of my scripts.

A major limitation of this is that any work where the author count does not match the affiliation count due to multiple affiliations must be excluded, as it is impossible to determine which affiliation corresponds to which author in these cases.

In [1]:
import datetime
import pandas as pd

In [2]:
file = r'' #put csv file path string here

In [11]:
institution = '' #put institution name here

In [ ]:
works = pd.read_csv(file, header=0)
df = pd.DataFrame(works)
df.dropna(subset='publication_year') # OpenAlex exports sometimes have random junk rows, usually this catches them

In [6]:
authorship_columns = ['authorships.author_position', 'authorships.institutions',
                      'authorships.countries', 'authorships.is_corresponding',
                      'authorships.raw_author_name', 'authorships.raw_affiliation_strings',
                      'authorships.affiliations', 'authorships.author.id',
                      'authorships.author.display_name', 'authorships.author.orcid']

In [13]:
for col in authorship_columns:
    df[col] = df[col].str.split('|')

In [14]:
# Dropping all rows where there are more affiliations than authors due to the issue of multiple authorships
mismatched_rows = df.apply(
    lambda row: len(row['authorships.author.display_name']) != len(row['authorships.affiliations'])
    if isinstance(row['authorships.author.display_name'], list) and isinstance(row['authorships.affiliations'], list)
    else False,
    axis=1
)

df = df[~mismatched_rows]

In [15]:
# Remove entries not containing the institution string
def filter_unaffiliated(row):

    if len(row['authorships.author.display_name']) != len(row['authorships.affiliations']):
        return pd.Series([None] * len(row), index=row.index)

    if isinstance(row['authorships.affiliations'], list):
        # Identify the indices where the institution string appears in any form
        keep_indices = [i for i, affiliation in enumerate(row['authorships.affiliations']) if institution in affiliation]

        # If no matching indices, drop the row
        if not keep_indices:
            return pd.Series([None] * len(row), index=row.index)

        # Filter each column in the authorship group based on `keep_indices`
        for col in authorship_columns:
            if isinstance(row[col], list):
                # Use list comprehension to only get valid indices that exist in the current list
                row[col] = [row[col][i] for i in keep_indices if i < len(row[col])]

    return row

In [16]:
df = df.apply(filter_unaffiliated, axis=1).dropna(how='all', subset=authorship_columns)

In [17]:
for col in authorship_columns:
    df[col] = df[col].apply(lambda x: '|'.join(x) if isinstance(x, list) else '')

In [23]:
filename = '' #put filename here

date = datetime.datetime.today().strftime('%Y-%m-%d')

In [24]:
df.to_csv(date + '_' + filename + '.csv', index=False)